In [8]:
import dao.MongoUsersSong as mus
import panda.SongUser as supd
# 展示操作一个用户的时候应该如何使用
mongomus = mus.UserSongRecom(user_id="1a78ac10b67911ed95dd00155dadb10b")
# mongomus.makeRecomUsersSetAnswer(limit="ONE")

In [9]:
# 获取recom中之前设置的数据
projections = {"_id": 0, "id": 1, "name": 1, "RecomUsers": 1}
doc = mongomus.rdb.findDocument(
    collection_name="recom", query=mongomus.query, projection=projections)
print(doc)
# 储存请求用户的名字
mongomus.user_name = doc[0]['name']
# 跟据数据库给的RecUsers来处理数据
list_users_id = []
list_users_id.append(mongomus.user_id)
# print(self.user_id, list_users_id)
# users_id 合集，之后会通过这个来形成x轴坐标
list_users_id.extend(doc[0]['RecomUsers'])
# 构成xy轴的内容源头
docs = mongomus.getSongsWithUserIdFromMongo(array=list_users_id)
# print(docs)

[{'id': '0bc2cc42a7be11edb7f600155daffd24', 'name': 'RuRuUIH', 'RecomUsers': ['0bc3112aa7be11edb7f600155daffd24', '0bc2d480a7be11edb7f600155daffd24', '0bc2dd40a7be11edb7f600155daffd24', '0bc305cca7be11edb7f600155daffd24', '0bc2e394a7be11edb7f600155daffd24', '0bc2d340a7be11edb7f600155daffd24', '0bc2f050a7be11edb7f600155daffd24', '0bc2dff2a7be11edb7f600155daffd24', '0bc31b5ca7be11edb7f600155daffd24', '0bc2e164a7be11edb7f600155daffd24', '0bc2d99ea7be11edb7f600155daffd24', '0bc2f10ea7be11edb7f600155daffd24', '0bc2dcaaa7be11edb7f600155daffd24', '0bc2ef9ca7be11edb7f600155daffd24', '0bc2e812a7be11edb7f600155daffd24', '0bc2d3c2a7be11edb7f600155daffd24', '0bc30838a7be11edb7f600155daffd24', '0bc31940a7be11edb7f600155daffd24', '0bc31f80a7be11edb7f600155daffd24', '0bc300f4a7be11edb7f600155daffd24', '0bc2fe6aa7be11edb7f600155daffd24', '0bc2e8b2a7be11edb7f600155daffd24', '0bc3216aa7be11edb7f600155daffd24', '0bc3185aa7be11edb7f600155daffd24', '0bc30c7aa7be11edb7f600155daffd24', '0bc31daaa7be11edb7f60

In [10]:
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongomus.makeXYMatric(docs=docs)
# print(data)
sudf = supd.SUPandas(datas=list(
    map(list, data.toarray())), items=mongomus.y, users=mongomus.x)
    

In [13]:
sudf.df

,RuRuUIH,4CRZrrs,5MpoD,p,9w6CypKgN,9qqrh,YH,MN,z,OXmegpgti,...,pQ2n0M,npwW8p7B,VQnw,KN,oqs8rJXaag,FO,DM7IN5IY,Fd,A,u6L4nw
229376,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
163848,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
415793161,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
327690,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29753363,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400162805,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
229365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
65528,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
548143098,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# 推荐过程
# dict_user_song = mongomus.makeRecomDcition(df_object=sudf, sign=-1)
similar = sudf.makeSimilarityBetweenUser()

/home/arki/.local/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [15]:
similar

,RuRuUIH,4CRZrrs,5MpoD,p,9w6CypKgN,9qqrh,YH,MN,z,OXmegpgti,...,pQ2n0M,npwW8p7B,VQnw,KN,oqs8rJXaag,FO,DM7IN5IY,Fd,A,u6L4nw
RuRuUIH,1.000000,0.004908,0.001538,0.001232,0.006952,0.000000,0.000000,0.005450,0.001972,0.003580,...,0.008696,0.003205,0.001304,0.004926,0.002398,0.004076,0.000000,0.000000,0.003995,0.000000
4CRZrrs,0.004908,1.000000,0.005671,0.009885,0.007070,0.005482,0.007650,0.005240,0.005459,0.004808,...,0.007401,0.006783,0.007679,0.009860,0.006446,0.009641,0.003824,0.003195,0.007772,0.003254
5MpoD,0.001538,0.005671,1.000000,0.004748,0.009973,0.005369,0.005333,0.012358,0.008032,0.010233,...,0.010516,0.006936,0.005958,0.002836,0.005581,0.004082,0.005701,0.006502,0.006042,0.005312
p,0.001232,0.009885,0.004748,1.000000,0.009494,0.006630,0.002188,0.008811,0.007709,0.005641,...,0.006606,0.005842,0.013805,0.012417,0.015523,0.007916,0.008696,0.002141,0.006066,0.005470
9w6CypKgN,0.006952,0.007070,0.009973,0.009494,1.000000,0.006243,0.005171,0.009244,0.009356,0.005397,...,0.007905,0.007401,0.003260,0.007087,0.009324,0.004177,0.010101,0.009156,0.011638,0.004119
9qqrh,0.000000,0.005482,0.005369,0.006630,0.006243,1.000000,0.008306,0.002398,0.006645,0.003205,...,0.007752,0.004161,0.009324,0.008830,0.001072,0.004802,0.006849,0.001597,0.004717,0.003289
YH,0.000000,0.007650,0.005333,0.002188,0.005171,0.008306,1.000000,0.002384,0.003284,0.007471,...,0.004391,0.001374,0.004614,0.006572,0.005353,0.003576,0.010929,0.004769,0.004689,0.003263
MN,0.005450,0.005240,0.012358,0.008811,0.009244,0.002398,0.002384,1.000000,0.002387,0.006003,...,0.008818,0.006303,0.005484,0.004374,0.003436,0.008475,0.001033,0.002328,0.006487,0.003567
z,0.001972,0.005459,0.008032,0.007709,0.009356,0.006645,0.003284,0.002387,1.000000,0.009636,...,0.005501,0.004138,0.001151,0.006579,0.011866,0.005981,0.001355,0.003180,0.004695,0.001631
OXmegpgti,0.003580,0.004808,0.010233,0.005641,0.005397,0.003205,0.007471,0.006003,0.009636,1.000000,...,0.008084,0.009515,0.003336,0.008871,0.007132,0.010327,0.013233,0.005214,0.008482,0.006376


In [16]:
# 使用平均数，对以上的相似度结果求平均，并取高于平均数的对象形成一个list
diction_user = sudf.makeTopNUsers(similar=similar,sign=1)
diction = sudf.makeRecomUserBySong(
            topN=diction_user, song_num=50)

In [17]:
# 程序过程中会有sign 来判断是去单个，还是返回所有的
dict_result = {mongomus.user_name: diction[mongomus.user_name]}

In [19]:
print(dict_result)

{'RuRuUIH': [229376, 35504129, 483278849, 29999108, 27770889, 327690, 22757394, 29753363, 516358164, 1303019546, 2588700, 26329117, 25657374, 22642721, 29491235, 551534627, 16957477, 1605671, 26558505, 294956, 39436333, 511213613, 1830469676, 1097777, 5406769, 327736, 2031674, 835642, 28377149, 16646207, 480624706, 553959497, 479281226, 17596491, 1386152022, 1456406618, 1411121242, 17563740, 1490124895, 245856, 426852453, 26083432, 28786793, 400162922, 464994415, 28786803, 21971064, 22724729, 18137221, 29999238]}


In [22]:
# 最后保存
# mongomus.saveUserSongRecomAnswer(diction=dict_result)
docs = mongomus.changeDataFormat(diction=dict_result, list_keys=list(dict_result))
# print(docs)
mongomus.wdb.writeDocument(
    docs=docs, collection_name="recom")


update coll RuRuUIH in recom
